In [1]:
#pip install mxnet-cu100
import mxnet as mx
import mxnet.gluon.nn as mxnn
from mxnet import nd
from mxnet import gluon, autograd
from mxnet.gluon.data.vision import transforms as mxT
import time, os, shutil


ModuleNotFoundError: No module named 'mxnet'

In [29]:
ctx = mx.cpu() # to change it to gpu -- mx.gpu()
print(mx.__version__)#1.4.1

1.4.1


In [30]:
def mx_conv3x3(out_planes, stride=1):
    " 3x3 convolution with padding "
    return mxnn.Conv2D(out_planes, kernel_size=3, strides=stride, padding=1)


class mxBasicBlock(mxnn.HybridBlock):
    expansion = 1

    def __init__(self, planes, stride=1, downsample=None, **kwargs):
        super(mxBasicBlock, self).__init__(**kwargs)
        self.conv1 = mx_conv3x3(planes, stride)
        self.bn1 = mxnn.BatchNorm()
        self.conv2 = mx_conv3x3(planes)
        self.bn2 = mxnn.BatchNorm()
        self.downsample = downsample
        self.stride = stride

    def hybrid_forward(self, F, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = F.relu(residual + out)

        return out
    
class mxResNet_Cifar(mxnn.HybridBlock):

    def __init__(self, block, layers, num_classes=10, **kwargs):
        super(mxResNet_Cifar, self).__init__(**kwargs)

        self.inplanes = 16
        self.conv1 = mxnn.Conv2D(
            16, kernel_size=3, strides=1, padding=1)
        self.bn1 = mxnn.BatchNorm()
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.avgpool = mxnn.AvgPool2D(8, strides=1)
        self.fc = mxnn.Dense(num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = mxnn.HybridSequential()
            downsample.add(
                mxnn.Conv2D(planes * block.expansion,
                          kernel_size=1, strides=stride)
            )

        layers = mxnn.HybridSequential()
        layers.add(block(planes, stride, downsample))
        for _ in range(1, blocks):
            layers.add(block(planes))

        return layers
    
def hybrid_forward(self, F, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = self.fc(x)

        return x


def mx_resnet20_cifar(**kwargs):
    model = mxResNet_Cifar(mxBasicBlock, [3, 3, 3], **kwargs)
    return model

In [31]:
%%time
mxnet = mx_resnet20_cifar()
mxnet.initialize(ctx = ctx)

Wall time: 28.9 ms


In [32]:
%%time
mxnet.hybridize()

Wall time: 1.99 ms


In [20]:
normalize = mxT.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

batch_size = 128

train_transfrom = mxT.Compose([
    mxT.RandomFlipLeftRight(),
    mxT.ToTensor(),
    normalize
])

val_transform = mxT.Compose([
    mxT.ToTensor(),
    normalize
])

mxtrainset = gluon.data.vision.datasets.CIFAR10(
    './data', train=True).transform_first(train_transfrom)
mxtrainloader = gluon.data.DataLoader(
    mxtrainset, batch_size=batch_size, shuffle=True, num_workers=2)

mxtestset = gluon.data.vision.datasets.CIFAR10(
    './data', train=False).transform_first(val_transform)
mxtestloader = gluon.data.DataLoader(
    mxtestset, batch_size=batch_size, shuffle=False, num_workers=2)

In [21]:
%%time
mxobjective = gluon.loss.SoftmaxCrossEntropyLoss()
mxoptimizer = gluon.Trainer(mxnet.collect_params(), 'adam', {'learning_rate': 0.001})

Wall time: 8.95 ms


In [22]:
for epoch in range(10):
    for features, labels in mxtrainloader:
        with autograd.record():
            output = mxnet(features.as_in_context(ctx))
            loss = mxobjective(output, labels.as_in_context(ctx))
        loss.backward()
        mxoptimizer.step(batch_size)
    print('Epoch:', epoch, 'done.')

NotImplementedError: 